<h1>Making game</h1>

In [ ]:
import pygame
import random
import math

board_w, board_h = 800, 600
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREY = (200, 200, 200)
RED = (255, 0, 0)
LIGHT_RED = (255, 204, 203)
GREEN = (0, 255, 0)
BLUE = (0, 0, 128)
DARK_GREY = (50, 50, 50)

global boardSize
global numPeople
global infectionChance
global infectFreq
global recoveryTime
global initInfected
global runLots
global lotsValues

boardSize = 0.50
numPeople = 20

infectionChance = 0.4
recoveryTime = 4000
initInfected = int(numPeople * 0.1)
infectFreq = 1000

runLots = False
lotsValues = []
lotsTimer = []

SPEED = 1

In [ ]:
# This is an input text field, it is not finished yet
class TextField():
    def __init__(self, x, y, width=140, height=32, activeColour=WHITE, passiveColour=GREY):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.activeColour = activeColour
        self.passiveColour = passiveColour
        self.textSurface = pygame.surface.Surface((self.width, self.height))
        
        self.surface = self.textSurface.get_rect(topleft=(self.x, self.y))        
        
        self.text = ""
        
        self.active = False
        
        textBoarder = pygame.Rect(self.x, self.y, self.width, self.height)
        
    def addText(self, char):
        self.text += char
    
    def delText(self):  
        if len(self.text) > 0:
            self.text = self.text[:-1]
        else:
            self.text = ""
            
    def draw(self):
        surface = self.textSurface.copy()
        
        if self.active:
            surface.fill(self.activeColour)
        else:
            surface.fill(self.passiveColour)
            
        screen.blit(surface, (self.x, self.y)) 
        self.surface = surface.get_rect(topleft=(self.x, self.y))

        
# This is a non-interactive text label
class Label():
    def __init__(self, text, colour, x, y, name="", fontType = 'freesansbold.ttf', fontSize = 20):
        self.text = text
        self.colour = colour
        self.x = x
        self.y = y
        self.name = name
        self.fontType = fontType
        self.fontSize = fontSize
        self.font = pygame.font.SysFont(self.fontType, self.fontSize)
    
        self.active = False
        self.draw()

    def draw(self):
        self.fontSurface = self.font.render(self.text, True, self.colour)
        screen.blit(self.fontSurface, (self.x, self.y))     
        self.surface = self.fontSurface.get_rect()

# This is an interactive button with text inside
class Button():
    def __init__(self, x, y, text, activeFn, name="", textColour=BLACK, width=140, height=32, fontType='freesansbold.ttf', fontSize=20):
        self.x = x
        self.y = y
        self.name = name
        self.width = width
        self.height = height
        self.text = text
        self.textColour = textColour
        self.fontType = fontType
        self.fontSize = fontSize
        
        self.active = False
        self.activeColour = WHITE
        self.passiveColour = GREY
        self.activeFn = activeFn

        
        # the debounce should mean the code is only called once per 
        # click, regardless of how long it is held down for        
        self.activeDebounce = False
        
        self.buttonSurface = pygame.surface.Surface((width, height))
        self.surface = self.buttonSurface.get_rect()
        
        #find mid point of button, and move a bit to the left
        self.textPos_x = self.x + self.width / 2 - self.width*0.1
        self.textPos_y = self.y + self.height / 2 - self.height*0.2
        
        self.label = Label(self.text, self.textColour, self.textPos_x, self.textPos_y)        
    
    # Do all the graphics first, then call the required function
    def draw(self):
        if self.active:
            self.buttonSurface.fill(self.activeColour)
        else:
            self.buttonSurface.fill(self.passiveColour)
        
        screen.blit(self.buttonSurface, (self.x, self.y))
        self.label.draw()   
        self.surface = self.buttonSurface.get_rect(topleft=(self.x, self.y))
        
        if self.active and self.activeDebounce != self.active:
            self.activeFn()
            
        self.activeDebounce = self.active
          
# This is an interactive slide with text on the right hand side
class Slider():
    def __init__(self, values, x, y, name = "", width=140, height=32):
        self.labelLoc = 50
        
        minV, currentV, maxV = values
        
        #used to blit the stuff onto
        self.surf_ = pygame.surface.Surface((width, height))
        
        self.minV = minV
        self.currentV = (currentV - minV) / (maxV - minV)
        self.maxV = maxV
        self.x = x
        self.y = y
        self.width = width
        
        self.name = name 

        if self.width <= self.labelLoc:
            raise ValueError("Width must be greater than 30")
        
        self.height = height

        self.active = False
        
        self.sliderBarSurf = pygame.surface.Surface((10, self.height))
        self.sliderBarSurf.fill(WHITE)
        
        self.label = Label(f"{self.currentV * (self.maxV - self.minV) + self.minV:1.1f}", WHITE, self.x + (self.width - 30), self.y)
        
        #used to check if clicked on
        self.surface = self.sliderBarSurf.get_rect()

    def move(self):
        mouse_x, mouse_y = pygame.mouse.get_pos()
        
        self.currentV = (mouse_x - self.x) / (self.width-self.labelLoc) 

        if self.currentV < 0.:
            self.currentV = 0.
        if self.currentV > 1.:
            self.currentV = 1.
            
        self.label.text = f"{self.currentV * (self.maxV - self.minV) + self.minV:1.1f}"
        
    def draw(self):        
        surface = self.surf_.copy()
        
        rangeBar = pygame.Rect(self.x, self.y+self.height//2, self.width - self.labelLoc, 4)    
        pygame.draw.rect(screen, GREY, rangeBar)
        
        position = (self.x + self.currentV * (self.width - self.labelLoc), self.y )
        self.surface = self.sliderBarSurf.get_rect(topleft=position)
        
        screen.blit(self.sliderBarSurf, position)
        self.label.draw()

In [ ]:
class Board():
    def __init__(self, x=50, y=50, maxWidth=500, maxHeight=300, minWidth=100, minHeight=60):
        self.x = x
        self.y = y
        
        self.maxWidth = maxWidth
        self.maxHeight = maxHeight
        self.minWidth = minWidth
        self.minHeight = minHeight
        
        self.width = (self.maxWidth - self.minWidth) * boardSize  + self.minWidth
        self.height = (self.maxHeight - self.minHeight) * boardSize + self.minHeight
        
        self.boardSurface = pygame.surface.Surface((self.width, self.height))
        
        self.run = "stopped"
        self.active = False #not used
        
        self.people = []
        
        self.graphValues = []
        self.timeSinceLastAppend = 0
        self.timeIntervals = []
        
        self.surface = pygame.Rect(self.x, self.y, self.width, self.height)
        
        pygame.draw.rect(screen,RED,self.surface, 3)

        
    def drawPeople(self):
        if len(self.people) == 0:
            infected = random.choices(range(numPeople), k=initInfected)
            
            for idx in range(numPeople):
                if idx in infected:
                    condition = "i"
                else:
                    condition = "s"
                self.people.append(Person(condition))
        else:
            for idx,p in enumerate(self.people):
                x = p.x * self.width + self.x 
                y = p.y * self.height + self.y
                
                if x + 10 > self.width + self.x: #push away from right border
                    x = self.width + self.x - 10
                if x - self.x < 10:  #push away from left boarder
                    x = self.x + 10
                if y + 10 > self.height + self.y: #push away from bottom border
                    y = self.height + self.y -10
                if y - self.y < 10:
                    y = self.y + 10    # push away from top border
                    
                if p.infectionRing:
                    pygame.draw.circle(screen, p.infectCircleColour, (x, y), p.infectR, width=1) 
                pygame.draw.circle(screen, p.colour, (x, y), 5)        
                
     
    def movePeople(self):
        
        timeForRing = 750//SPEED # amount of time it takes for the infeciton ring to max out
        currentTime = pygame.time.get_ticks()
        
        valuesForGraph = [0,0,0]
        
        for idx,p in enumerate(self.people):   
            
            if p.status == "s":
                valuesForGraph[0] += 1
            elif p.status == "i":
                valuesForGraph[1] += 1
            elif p.status == "r":
                valuesForGraph[2] += 1
            
            # it is easier for my brain to convert x,y 
            # positions from units of 0-1 to units 0-width/height
            x = p.x * self.width + self.x             
            y = p.y * self.height + self.y
            
            x_ = p.x_
            y_ = p.y_
                                    
            x += x_
            y += y_
            
            p.x = (x - self.x) / self.width
            p.y = (y - self.y) / self.height            
            
            # check for collision with border            
            if (x + 10 > self.width + self.x) or (x - self.x < 10):
                p.x_ = -1*p.x_ 

            if (y + 10 > self.height + self.y) or (y - self.y < 10):
                p.y_ = -1*p.y_
                
            if p.status == "i":
                if (currentTime - p.lastInfectionAttempt) > (infectFreq//SPEED):
                    p.lastInfectionAttempt = currentTime
                    p.infectionRing = True
                elif (currentTime - p.lastInfectionAttempt) > (timeForRing//SPEED) and p.infectionRing:
                    p.infectR = 0
                    p.infectionRing = False                    

                if p.infectionRing:
                    percentComplete = (currentTime - p.lastInfectionAttempt) / (timeForRing//SPEED)
                    p.infectR = p.maxInfectR * percentComplete
        
        tots = valuesForGraph[0] + valuesForGraph[1] + valuesForGraph[2]
        lastTime = currentTime - self.timeSinceLastAppend
        if tots > 0 and (lastTime)>(100//SPEED): #first time through there might be no people
            if (lastTime)>200:
                assert 3==2,f"Skkipped an update!! {lastTime}"
            self.graphValues.append(valuesForGraph)
            self.timeIntervals.append(lastTime)
            self.timeSinceLastAppend = currentTime

    
    def infectPeople(self):
        #want to see if the infected people are near a suspiable person
        for infectedIdx, infectedP in enumerate(self.people):
            if infectedP.status != "i":
                continue
                
            if infectedP.infectionRing == False: # isn't currently infecting people
                continue
                
            for otherIdx, otherP in enumerate(self.people):
                if otherP.status != "s":
                    continue
                
                x = (infectedP.x - otherP.x) * self.width
                y = (infectedP.y - otherP.y) * self.height
                z = (x**2 + y**2)**0.5                
                
                if abs(infectedP.infectR - z) < 5:
                    otherP.gotInfected()
    
    def recoverPeople(self):
        currentTime = pygame.time.get_ticks()
        self.run = "stopped"
        
        for idx,p in enumerate(self.people):
            if p.status == "i":
                self.run = "running" #keep the sim going while there are infected people
                if (currentTime - p.initInfectionTime) > (recoveryTime//SPEED):
                    p.status = "r"
                    p.updateColour()
                    p.infectR = 0
                    p.infectionRing = False
        
        if self.run == "stopped" and runLots:
            lotsValues.append(self.graphValues)
            lotsTimer.append(self.timeIntervals)
            if len(lotsValues) < 150:
                self.run = "init"
                    
    def update(self):
        self.movePeople()
        self.infectPeople()
        self.recoverPeople()

    
    def draw(self):
        pygame.draw.rect(screen,RED,self.surface, 3)   
        
        pygame.draw.line(screen, GREY, (self.x, board_h-50), (self.x,board_h-200), width=1)
        pygame.draw.line(screen, GREY, (self.x, board_h-50), (self.maxWidth-self.x,board_h-50), width=1)

        
        if self.run == "running":
            self.drawPeople()            
        elif self.run == "init":
            self.people = []
            self.graphValues = []
            self.timeIntervals = []            
            self.drawPeople()
            self.run = "running"
        else:
            self.width = (self.maxWidth - self.minWidth) * boardSize  + self.minWidth
            self.height = (self.maxHeight - self.minHeight) * boardSize + self.minHeight
            self.surface = pygame.Rect(self.x, self.y, self.width, self.height)


        self.updateGraph()
            
    def updateGraph(self):
        numRects = len(self.graphValues)

        if numRects > 0:
            xMin = self.x + 10
            yMin = board_h-50 - 10
            xMax = self.maxWidth-self.x - 10
            yMax = board_h-200 + 10

            rectWidth = (xMax - xMin) / numRects
            rectHeight = (yMin - yMax)

            if rectWidth < 1.5:
                self.run = "stopped"

            for idx, gv in enumerate(self.graphValues):

                s, i, r = gv
                total = s+i+r
                s /= total
                i /= total
                r /= total

                rRect = pygame.Rect(int(xMin + idx*rectWidth -1), yMax, rectWidth+1, rectHeight*r)
                sRect = pygame.Rect(int(xMin + idx*rectWidth -1), yMax+rectHeight*r, rectWidth+1, rectHeight*s)
                iRect = pygame.Rect(int(xMin + idx*rectWidth -1), int(yMax+rectHeight*(r+s)-1), rectWidth+1, int(rectHeight - (int(rectHeight*(r+s)-1))))

                pygame.draw.rect(screen,GREY,rRect)
                pygame.draw.rect(screen,GREEN,sRect)
                pygame.draw.rect(screen,RED,iRect)


In [ ]:
class Person():
    def __init__(self, status):
        self.status = status
        
        # Position
        self.x = random.random()
        self.y = random.random()   
        
        # Velocity
        # We want the velocity vector to be of length 1
        self.x_ = random.random()
        self.y_ = random.random()
        z = (self.x_**2 + self.y_**2)**0.5        
        self.x_ /= (z + 1e-10)
        self.y_ /= (z + 1e-10)
        
        # probably a very silly check
        z = (self.x_**2 + self.y_**2)**0.5        
        assert z >= 0.9 and z <= 1.1, f"I failed at math"
        
        #once normaliesd, lets make it go in 2 directions
        self.x_ = (self.x_ * 2) - 1
        self.y_ = (self.y_ * 2) - 1        
        
        # infection ring radius
        self.infectR = 0
        self.infectCircleColour = LIGHT_RED
        self.maxInfectR = 35 #maximum size of infection ring
        self.infectionRing = False #is the ring active?
        
        if self.status == "i":
            self.lastInfectionAttempt = pygame.time.get_ticks() - (infectFreq//SPEED) * random.random() #first ping will occur at a random time
            self.initInfectionTime = pygame.time.get_ticks()
        else:
            self.lastInfectionAttempt = 0
            self.initInfectionTime = 0
        
        self.updateColour()
        
    def gotInfected(self):

        if self.status == "s":
            chance = random.random()
            if chance <= infectionChance:
                self.status = "i"
                self.lastInfectionAttempt = pygame.time.get_ticks() - (infectFreq//SPEED) * random.random() #first ping will occur at a random time
                self.initInfectionTime = pygame.time.get_ticks()
                self.updateColour()
    
    def updateColour(self):
        if self.status == "s":
            self.colour = GREEN
        elif self.status == "i":
            self.colour = RED
        else:
            self.colour = DARK_GREY

In [ ]:
def runNow():
    pygame.time.set_timer(moveEvent, 20//SPEED) #every 20ms/
    items["board"][0].timeSinceLastAppend = pygame.time.get_ticks()
    
def stopNow():
    pygame.time.set_timer(moveEvent, 0)

In [ ]:
def setupScreen():
    startY = 20
    numPeopleText = Label("Number of people", WHITE, 600, startY)
    startY += 20
    numPeopleSlider = Slider((5,25,50), x=600, y=startY, name="numPeople")

    startY += 40
    perInfectedText = Label("Percentage infected", WHITE, 600, startY)   
    startY += 20
    infectedSlider = Slider((0,10,100), x=600, y=startY, name="perInfected")
    
    startY += 40
    areaSizeText = Label("Size of area", WHITE, 600, startY)
    startY += 20
    areaSizeSlider = Slider((0,50,100), x=600, y=startY, name="boardSize")
    
    startY += 40    
    infectFreqText = Label("Days per infection attempt", WHITE, 600, startY)
    startY += 20
    infectFreqSlider = Slider((1,2,10), x=600, y=startY, name="infectFreq")
    
    startY += 40    
    infectChanceText = Label("Chance of infection", WHITE, 600, startY)
    startY += 20
    infectChancelider = Slider((1,40,100), x=600, y=startY, name="infectChance")    

    startY += 40
    recoveryPeriodText = Label("Days for recovery", WHITE, 600, startY)
    startY += 20
    recoveryPeriodSlider = Slider((1,4,10), x=600, y=startY, name="recoveryTime")   
    
    goButton = Button(x=600, y=450, text="Run", activeFn=runNow, name="run")
    stopButton = Button(x=600, y=500, text="Stop", activeFn=stopNow, name="stop")
    
    runLotsbutton = Button(x=600, y=550, text="Run LOTS!", activeFn=runNow, name="runLots")
    
    board = Board()

    items = {
        "text": [numPeopleText, perInfectedText, areaSizeText, infectFreqText, infectChanceText, recoveryPeriodText],
        "slider": [numPeopleSlider, infectedSlider, areaSizeSlider, infectFreqSlider, infectChancelider, recoveryPeriodSlider],
        "button": [goButton, stopButton, runLotsbutton],
        "board": [board,]
    }
    
    return items

running = True
    
pygame.init()
pygame.font.init()
pygame.display.set_caption("Virus Simulator!!!11one!!")
clock = pygame.time.Clock()
clock.tick(30)
screen = pygame.display.set_mode((board_w, board_h))
screen.fill(BLACK)

items = setupScreen()

moveEvent = pygame.USEREVENT+1

pygame.time.set_timer(moveEvent, 0)

activeKeys = ["slider", "button"]

while running:
    # event handling, gets all event from the event queue
    for event in pygame.event.get():

        if event.type == pygame.QUIT:
            running = False
            
        if event.type == pygame.MOUSEBUTTONDOWN:
            pos = pygame.mouse.get_pos()
            for ak in activeKeys:
                for i in items[ak]:
                    if i.surface.collidepoint(pos):
                        i.active = True
            
        if event.type == pygame.MOUSEBUTTONUP:
            for ak in activeKeys:
                for i in items[ak]:
                    i.active = False
                    
        if event.type == moveEvent: 
            items["board"][0].update()

    screen.fill(BLACK)

    for i in items["slider"]:
        if i.active:
            i.move()  
            
        if i.name == "boardSize":
            boardSize = i.currentV
        elif i.name == "numPeople":
            numPeople = int(i.currentV * (i.maxV - i.minV) + i.minV)
        elif i.name == "perInfected":
            initInfected = int(i.currentV * numPeople)
        elif i.name == "infectFreq":
            infectFreq = 1000 * (i.currentV * (i.maxV - i.minV) + i.minV) //SPEED# infectionAttempts / second
        elif i.name == "infectChance":
            infectionChance = i.currentV
        elif i.name == "recoveryTime":   
            recoveryTime = 1000 * (i.currentV * (i.maxV - i.minV) + i.minV) //SPEED#seconds
            
    for i in items["button"]:
        if i.active:
            if i.name == "run":
                items["board"][0].run = "init"
            elif i.name == "stop":
                items["board"][0].run = "stopped"
            elif i.name == "runLots":
                runLots = True
                items["board"][0].run = "init"

    if items["board"][0].run == "stopped":
        stopNow()

    for k,l in items.items():
        for i in l:
            i.draw()
        
    pygame.display.update()

pygame.quit()

<h1>Making graphics</h1>
<b>Only useful if you click "run lots"</b>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint


lV = lotsValues.copy()
lTimer = lotsTimer.copy()
lVarray = []
lTarray = []

for timeStep in range(200):
    vals = []
    tims = []
    for idx in range(len(lV)):
        if len(lV[idx]) > timeStep:
            vals.append(lV[idx][timeStep])
            tims.append(lTimer[idx][timeStep])
    if len(vals)>0:
        vals = np.array(vals)
        lVarray.append(np.sum(vals,0)/len(lV))   
        lTarray.append(np.mean(tims))
lVarray = np.array(lVarray)
s = lVarray[10:,0]
i = lVarray[10:,1]
r = lVarray[10:,2]
t = []
for x in lTarray[10:]:
    if len(t) == 0:
        t.append(x)
    else:
        t.append(x+t[-1])
t = np.array(t)/1000

In [ ]:

def deriv(y, t, beta, gamma, P0):
    S, I, R = y
    dSdt = -beta * S * I / P0
    dIdt = beta * S * I / P0 - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt



In [ ]:
y0 = [s[0], i[0], 0]   
minError = 1e30
values = i.copy()
values = np.array(values)


for infectionRate in np.arange(0.21, 2.00, 0.01):
    for recoveryRate in np.arange(1,10,0.1):
        ret = odeint(deriv, y0, t, args=(infectionRate, 1/recoveryRate, s[0]+i[0]))
        S, I, R = ret.T

        # Some of squared error
        error = np.sum((values - I[:len(values)])**2) 
        if error < minError:
            minError = error
            goodVal = [infectionRate, recoveryRate]            

ret = odeint(deriv, y0, t, args=(goodVal[0], 1/goodVal[1], s[0]+i[0]))
S, I, R = ret.T

In [ ]:

x = t[:len(i)]

plt.figure(figsize=(10,10), facecolor='white')
plt.bar(x, i, color='r', width=1, label = 'infected')
plt.bar(x, s, bottom=i, color='g', width=1, label ='susceptible')
plt.bar(x, [1,]*((s[0]+i[0])-(i+s)), bottom=i+s, color='gray', width=1, label='recovered')
plt.xlim(x[0],x[-1])
plt.ylim(0,(s[0]+i[0]))
plt.plot(x, I[:len(x)], 'k', label='model')
plt.legend(fontsize = 16)
plt.title(f"Infection rate {goodVal[0]:2.3f}, Recovery rate {goodVal[1]:1.1f}", fontsize=16)
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)


plt.show()